In [2]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.spatial import distance_matrix
from scipy.sparse.csgraph import minimum_spanning_tree
import os
import subprocess

# Global settings
plt.rcParams.update({
    'font.family': 'Arial',
    'font.size': 14,
    'figure.dpi': 150,
    'axes.titlesize': 18,
    'axes.titleweight': 'bold',
    'axes.labelweight': 'bold',
    'savefig.facecolor': 'none'  # crucial: no white background
})

# =============== Generate 3D normal points ===============
np.random.seed(42)
n = 100
mu = np.array([0, 0, 0])
SIGMA = np.array([[3, 1, 1.5],
                  [1, 2, 0.5],
                  [1.5, 0.5, 2]])

data = np.random.multivariate_normal(mu, SIGMA, size=n)
x, y, z = data[:, 0], data[:, 1], data[:, 2]
points = np.vstack((x, y, z)).T

# Compute MST
dist_matrix = distance_matrix(points, points)
mst_sparse = minimum_spanning_tree(dist_matrix).tocoo()
mst_edges = [(i, j) for i, j in zip(mst_sparse.row, mst_sparse.col)]

# Create figure and axis
fig = plt.figure(figsize=(10, 8))
fig.patch.set_alpha(0.0)  # transparent figure
ax = fig.add_subplot(111, projection='3d')

scatter = ax.scatter(x, y, z,
                     c=z, cmap='viridis', s=20, alpha=0.8,
                     edgecolors='w', linewidth=0.2)

lines = []
for i, j in mst_edges:
    xi, yi, zi = points[i]
    xj, yj, zj = points[j]
    line, = ax.plot([xi, xj], [yi, yj], [zi, zj], color='red', linewidth=0.8, alpha=0.7)
    lines.append(line)

xlim = (mu[0] - 4*np.sqrt(SIGMA[0,0]), mu[0] + 4*np.sqrt(SIGMA[0,0]))
ylim = (mu[1] - 4*np.sqrt(SIGMA[1,1]), mu[1] + 4*np.sqrt(SIGMA[1,1]))
zlim = (mu[2] - 4*np.sqrt(SIGMA[2,2]), mu[2] + 4*np.sqrt(SIGMA[2,2]))

ax.set_xlim(xlim)
ax.set_ylim(ylim)
ax.set_zlim(zlim)

# Output folder for frames
frames_dir = "frames"
if not os.path.exists(frames_dir):
    os.makedirs(frames_dir)

print("Generating frames...")
# Generate frames with transparent background
for i, angle in enumerate(range(0, 360, 2)):
    ax.view_init(elev=25, azim=angle)
    plt.savefig(f"{frames_dir}/frame_{i:03d}.png", dpi=100, transparent=True)

plt.close(fig)

# =============== Assemble MOV with FFmpeg ===============
mov_path = "mu000_3d_normal_with_mst_transparent.mov"
print("Assembling MOV with FFmpeg...")

cmd = [
    "ffmpeg", "-y",
    "-framerate", "15",
    "-i", f"{frames_dir}/frame_%03d.png",
    "-c:v", "prores_ks",
    "-profile:v", "4",
    "-pix_fmt", "yuva444p10le",
    mov_path
]
subprocess.run(cmd, check=True)

print(f"MOV file with transparent background saved to: {mov_path}")

Generating frames...
Assembling MOV with FFmpeg...


ffmpeg version 7.1.1 Copyright (c) 2000-2025 the FFmpeg developers
  built with Apple clang version 17.0.0 (clang-1700.0.13.3)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.1.1_3 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex

MOV file with transparent background saved to: mu000_3d_normal_with_mst_transparent.mov


[out#0/mov @ 0x600003ae4000] video:49891KiB audio:0KiB subtitle:0KiB other streams:0KiB global headers:0KiB muxing overhead: 0.003517%
frame=  180 fps=114 q=-0.0 Lsize=   49893KiB time=00:00:12.00 bitrate=34060.4kbits/s speed=7.61x    
